In [ ]:
!pip install transformers

In [ ]:
!pip install dataset

In [ ]:
!pip install soundfile
!pip install librosa

In [ ]:
!pip install git+https://github.com/huggingface/transformers

In [ ]:
!pip install gradio
!pip install -U typing_extensions
!pip install -U pydantic

In [1]:
from huggingface_hub import notebook_login

notebook_login()

## Speech to Speech Translation

### Speech Translation

In [2]:
import torch
from transformers import pipeline
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-base", device=device)

In [3]:
device

'cuda:0'

In [4]:
from datasets import load_dataset
dataset=load_dataset("facebook/voxpopuli", "it", split="validation", streaming=True)
sample = next(iter(dataset))

In [5]:
import IPython
IPython.display.Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

In [6]:
def translate(audio):
    outputs = pipe(audio, max_new_tokens=256, generate_kwargs={"task":"translate"})
    return outputs["text"]

In [7]:
translate(sample["audio"].copy())

' psychological and social. I think that it is a very important step in the construction of a juridical space of freedom, circulation and protection of rights.'

### Text-to-speech

In [8]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [9]:
model.to(device)
vocoder.to(device)

SpeechT5HifiGan(
  (conv_pre): Conv1d(80, 512, kernel_size=(7,), stride=(1,), padding=(3,))
  (upsampler): ModuleList(
    (0): ConvTranspose1d(512, 256, kernel_size=(8,), stride=(4,), padding=(2,))
    (1): ConvTranspose1d(256, 128, kernel_size=(8,), stride=(4,), padding=(2,))
    (2): ConvTranspose1d(128, 64, kernel_size=(8,), stride=(4,), padding=(2,))
    (3): ConvTranspose1d(64, 32, kernel_size=(8,), stride=(4,), padding=(2,))
  )
  (resblocks): ModuleList(
    (0): HifiGanResidualBlock(
      (convs1): ModuleList(
        (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
        (2): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(5,), dilation=(5,))
      )
      (convs2): ModuleList(
        (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (2): Conv1d(256, 256, ker

In [10]:
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

Reusing dataset cmu-arctic-xvectors (/root/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/a62fea1f9415e240301ea0042ffad2a3aadf4d1caa7f9a8d9512d631723e781f)


In [11]:
def synthesize(text):
    inputs = processor(text=text, return_tensors="pt")
    speech = model.generate_speech(inputs["input_ids"].to(device), speaker_embeddings.to(device), vocoder=vocoder)
    return speech.cpu()

In [12]:
speech = synthesize("He's a dude, I'm a dude, she's a dude. We're all dudes yeah!")
IPython.display.Audio(speech, rate=16_000)

### Create a STST demo

In [13]:
import numpy as np
target_dtype = np.int16
max_range = np.iinfo(target_dtype).max

def speech_to_speech_translation(audio):
    translated_text = translate(audio)
    synthesized_speech = synthesize(translated_text)
    synthesized_speech = (synthesized_speech.numpy() * max_range).astype(np.int16)
    return 16000, synthesized_speech

sampling_rate, synthesized_speech = speech_to_speech_translation(sample["audio"])
IPython.display.Audio(synthesized_speech, rate=sampling_rate)

In [ ]:
import gradio as gr

demo = gr.Blocks()

mic_translate = gr.Interface(fn=speech_to_speech_translation,
                             inputs=gr.Audio(source="microphone", type="filepath"),
                             outputs=gr.Audio(label="Generated Speech", type="numpy"),
                            )
file_translate = gr.Interface(fn=speech_to_speech_translation,
                              inputs=gr.Audio(souce="upload", type="filepath"),
                              outputs=gr.Audio(label="Generated Speech", type="numpy"),
                             )
                                              
with demo:
    gr.TabbedInterface([mic_translate, file_translate], ["Microphone", "Audio File"])
    
demo.launch(debug=True, share=True) 

/tmp/ipykernel_877/3509222265.py:10: GradioUnusedKwargWarning: You have unused kwarg parameters in Audio, please remove them: {'souce': 'upload'}
  inputs=gr.Audio(souce="upload", type="filepath"),
/usr/local/lib/python3.9/dist-packages/gradio/blocks.py:934: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://34732174917d2e0fa5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/gradio/routes.py", line 439, in run_predict
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.9/dist-packages/gradio/blocks.py", line 1384, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.9/dist-packages/gradio/blocks.py", line 1089, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.9/dist-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.9/dist-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.9/dist-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "/usr/local/lib/python3.9/dist-packages/gradio/utils.py", line 700, in wrapper
    response = f(*args, **kwargs)
  File "/tm